In [1]:
import numpy as np

In [4]:
import torch
a = torch.ones((10,3,5,5)).cuda()
print(a[:,0].sum())

tensor(250., device='cuda:0')


In [13]:
a = np.zeros((100,100))
a[:2,:] += 1
print(a[97,3])
print(a)

0.0
[[1. 1. 1. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [17]:
import torch
a = torch.from_numpy(np.zeros((100,3,10,10)))
b = torch.from_numpy(np.ones((100,1,10,10)))
c = torch.cat((a,b),1)
c.size()


torch.Size([100, 4, 10, 10])

In [21]:
print(c[:,-1].reshape(100,1,10,10).size())

torch.Size([100, 1, 10, 10])


In [22]:
print(len(c))

100


In [9]:
from matplotlib import pyplot as plt
from celluloid import Camera

fig = plt.figure()
camera = Camera(fig)
for i in range(10):
    plt.plot([i] * 10)
    camera.snap()
animation = camera.animate()
animation.save('celluloid_minimal.gif', writer = 'imagemagick')

MovieWriter imagemagick unavailable; using Pillow instead.


In [1]:
import numpy as np

In [4]:
a = np.load("loss_history.npy")

In [5]:
print(a)

[10.95741367 10.20103359 10.15862465 ...  4.66714287  4.63087654
  5.13131332]


In [9]:
from learning.CoordConvModule import *
from torch import nn
import numpy as np

In [26]:
class CCenc(nn.Module):
    def __init__(self):
        super(CCenc, self).__init__()
        # Input size: [batch, 3, 100, 100]
        # Output size: [batch, 3, 100, 100]
        self.encoder = nn.Sequential(
            CoordConv(in_channels=3, out_channels=32, kernel_size = 3, stride = 1, padding = 1),
            CoordConv(in_channels=32, out_channels=32, kernel_size = 5, stride = 1, padding = 2),
            CoordConv(in_channels=32, out_channels=3, kernel_size = 3, stride = 1, padding = 1),
        )
    def forward(self, x):
        x = self.encoder(x)
        print(x.size())
        return x

In [27]:
model = CCenc()

In [28]:
a = torch.tensor(np.ones((1,3,100,100))).float()
aa = model(a)

torch.Size([1, 3, 100, 100])


In [2]:
import torch
import numpy as np
from torch import nn
from learning.CoordConvModule import *

In [11]:
class PrintLayer(nn.Module):
    def __init__(self):
        super(PrintLayer, self).__init__()
    
    def forward(self, x):
        # Do your print / debug stuff here
        print(x.size())
        return x

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # Input size: [batch, 3, 100, 100]
        # Output size: [batch, 3, 100, 100]
        self.encoder = nn.Sequential(
            CoordConv(3, 64, kernel_size = 1, stride=1),
            nn.LeakyReLU(),
            PrintLayer(),
            nn.Conv2d(64, 64, kernel_size = 5, stride=1, padding=1),
            # nn.BatchNorm2d(64),
            nn.LeakyReLU(),
            PrintLayer(),
            nn.Conv2d(64, 64, 7, stride=1, padding=1),
            # nn.BatchNorm2d(6
            nn.LeakyReLU(),
            PrintLayer(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 64, 7, stride=1, padding=1),
            nn.LeakyReLU(),
            PrintLayer(),
            nn.ConvTranspose2d(64, 64, 5, stride=1, padding=1),
            nn.LeakyReLU(),
            PrintLayer(),
            nn.Conv2d(64, 3, 1, stride=1),
            PrintLayer(),
        )

    def forward(self, x):
        # print(x.size())
        # wall = x[:, -1]
        # print(wall.size())
        x = self.encoder(x)
        x = self.decoder(x)
        # print(x.size())
        # x = torch.cat((x, wall.reshape(len(x), 1, width_fig, width_fig)), 1)
        # print(x.size())
        return x

In [12]:
model = Autoencoder()

In [13]:
model.eval()

Autoencoder(
  (encoder): Sequential(
    (0): CoordConv(
      (addcoords): AddCoords()
      (conv): Conv2d(5, 64, kernel_size=(1, 1), stride=(1, 1))
    )
    (1): LeakyReLU(negative_slope=0.01)
    (2): PrintLayer()
    (3): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (4): LeakyReLU(negative_slope=0.01)
    (5): PrintLayer()
    (6): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1), padding=(1, 1))
    (7): LeakyReLU(negative_slope=0.01)
    (8): PrintLayer()
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(64, 64, kernel_size=(7, 7), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.01)
    (2): PrintLayer()
    (3): ConvTranspose2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (4): LeakyReLU(negative_slope=0.01)
    (5): PrintLayer()
    (6): Conv2d(64, 3, kernel_size=(1, 1), stride=(1, 1))
    (7): PrintLayer()
  )
)

In [14]:
a = torch.Tensor(np.ones((1,3,100,100)))
model(a)

torch.Size([1, 64, 100, 100])
torch.Size([1, 64, 98, 98])
torch.Size([1, 64, 94, 94])
torch.Size([1, 64, 98, 98])
torch.Size([1, 64, 100, 100])
torch.Size([1, 3, 100, 100])


tensor([[[[ 0.0077,  0.0080,  0.0097,  ...,  0.0093,  0.0075,  0.0065],
          [ 0.0079,  0.0108,  0.0127,  ...,  0.0077,  0.0056,  0.0053],
          [ 0.0081,  0.0091,  0.0131,  ...,  0.0076,  0.0065,  0.0065],
          ...,
          [ 0.0052,  0.0048,  0.0077,  ...,  0.0049,  0.0077,  0.0063],
          [ 0.0055,  0.0053,  0.0067,  ...,  0.0051,  0.0081,  0.0068],
          [ 0.0061,  0.0061,  0.0067,  ...,  0.0050,  0.0069,  0.0072]],

         [[-0.0594, -0.0583, -0.0585,  ..., -0.0585, -0.0580, -0.0573],
          [-0.0603, -0.0592, -0.0605,  ..., -0.0567, -0.0572, -0.0584],
          [-0.0582, -0.0565, -0.0569,  ..., -0.0573, -0.0568, -0.0593],
          ...,
          [-0.0569, -0.0580, -0.0548,  ..., -0.0556, -0.0559, -0.0588],
          [-0.0584, -0.0590, -0.0560,  ..., -0.0580, -0.0582, -0.0599],
          [-0.0569, -0.0576, -0.0561,  ..., -0.0576, -0.0575, -0.0583]],

         [[ 0.0783,  0.0784,  0.0788,  ...,  0.0809,  0.0815,  0.0811],
          [ 0.0779,  0.0769,  